In [1]:
import sys
import os, os.path

sys.path.append(os.path.join(os.getcwd() ,'/modules'))
root_path = "C:/git/Springboard-Public/Capstone Project 2/"
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/My Drive/Capstone Project 2/"

print('Current Working Dir: ', os.getcwd())
print('Root Path: ', root_path)

# We need to set the working directory since we are using relative paths from various locations
if os.getcwd() != root_path:
  os.chdir(root_path)

Current Working Dir:  C:\git\Springboard-Public\Capstone Project 2\notebooks
Root Path:  C:/git/Springboard-Public/Capstone Project 2/


In [2]:
from modules.lib.StandardTraining import *
from modules.models.ResNet import ResNet_GrayScale_12_Out

from torchsummary import summary

%matplotlib inline

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(f'Working on device={device}')

Working on device=cuda


In [4]:
class Nets:
    
    def __init__(self):
        self.cur_idx = 0
        self.nets = []
        layers = [18] # ResNet Laysers 18, 34, 50, 101, 152
        lrs = [1e-5]
        batch_sizes = [32]
        pos_wights = [False, True]

        combinations = [(l, lr, bs, w) for l in layers for lr in lrs for bs in batch_sizes for w in pos_wights] 

        for combination in combinations:
            layers, lr, batch_size, w = combination   
            name = 'ResNet' + str(layers) + f'\n    learning rate:{lr}\n    batch size:{batch_size}\n    pos weights:{w}'
            self.nets.append((name, layers, lr, batch_size, w))
            
    def __iter__(self):
        for name, layers, lr, batch_size, w in self.nets:
            net = ResNet_GrayScale_12_Out(layers=layers)
            net = nn.DataParallel(net)
            net.to(device)
            yield ModelLoop.getConfigObject(name, net, 
                                            learning_rate=lr, 
                                            batch_size=batch_size,
                                            use_positivity_weights = w)
    
    def __len__(self):
        return len(self.nets)

In [5]:
modelLoop = ModelLoop(number_images=25000, 
                      default_batch_size=32, 
                      default_learning_rate = 1e-4,
                      num_epochs=4,
                      device=device, 
                      nets=Nets())


Positive Weights used in BCEWithLogitsLoss:


Enlarged_Cardiomediastinum    18.538086
Cardiomegaly                   8.089959
Lung_Opacity                   1.185602
Lung_Lesion                   22.789536
Edema                          3.265885
Consolidation                 16.143959
Pneumonia                     35.442623
Atelectasis                    5.373686
Pneumothorax                  11.319581
Pleural_Effusion               1.934438
Pleural_Other                 79.028000
Fracture                      22.236934
dtype: float64

In [6]:
modelLoop.train()

▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆

ResNet18
    learning rate:1e-05
    batch size:32
    pos weights:False

▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Number of Training Images: 20,047
Number of Validation Images: 4,953
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [1/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 213.5034         
Time of Completion: 2020-07-25 20:18:51.502172          
Training Time: 0:03:35.014931          
Validation Time: 0:00:29.751555
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [2/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 191.7023         
Time of Completion: 2020-07-25 20:22:09.771869          
Training Time: 0:02:52.491610          
Validation Time: 0:00:25.776080
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [3/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔

,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,2260,0.000442,0.125000,0.000882,0.743905,0.252857
4,Edema,4757,0.245953,0.582379,0.345847,0.779443,0.497106
5,Consolidation,1164,0.000000,0.000000,0.000000,0.671517,0.100560
7,Atelectasis,3153,0.000000,0.000000,0.000000,0.624384,0.220175
9,Pleural_Effusion,6795,0.486093,0.645495,0.554567,0.785310,0.625783



VALIDATION

▔▔▔▔▔▔▔


,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,511,0.003914,0.181818,0.007663,0.742477,0.236845
4,Edema,1171,0.144321,0.599291,0.232622,0.765494,0.473575
5,Consolidation,307,0.000000,0.000000,0.000000,0.633222,0.097769
7,Atelectasis,806,0.000000,0.000000,0.000000,0.610102,0.215608
9,Pleural_Effusion,1734,0.307382,0.718329,0.430533,0.793102,0.648567








▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆

ResNet18
    learning rate:1e-05
    batch size:32
    pos weights:True

▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Number of Training Images: 20,007
Number of Validation Images: 4,993
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [1/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 739.6222         
Time of Completion: 2020-07-25 20:34:07.719909          
Training Time: 0:03:48.698551          
Validation Time: 0:00:33.388780
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [2/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 719.7512         
Time of Completion: 2020-07-25 20:38:27.734975          
Training Time: 0:03:48.809528          
Validation Time: 0:00:31.204540
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [3/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔

,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,2201,0.655157,0.171259,0.271538,0.679520,0.186644
4,Edema,4690,0.727719,0.383828,0.502577,0.747657,0.432856
5,Consolidation,1167,0.629820,0.093834,0.163333,0.664350,0.103330
7,Atelectasis,3139,0.618031,0.193073,0.294229,0.596019,0.201584
9,Pleural_Effusion,6818,0.680405,0.506718,0.580855,0.729315,0.547579



VALIDATION

▔▔▔▔▔▔▔


,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,570,0.638596,0.194861,0.298605,0.698294,0.212897
4,Edema,1238,0.664782,0.438232,0.528241,0.762720,0.480525
5,Consolidation,304,0.661184,0.087888,0.155152,0.637219,0.088783
7,Atelectasis,820,0.628049,0.205753,0.309961,0.596219,0.210088
9,Pleural_Effusion,1711,0.680304,0.541144,0.602796,0.757209,0.583488
